In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#install CATBOOST
pip install catboost

     |████████████████████████████████| 62.9MB 37kB/s 


In [81]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
import seaborn as seabornInstance 
from catboost import CatBoostRegressor
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error 
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_absolute_error,mean_squared_error, r2_score
import lightgbm as lgb
# dataset = pd.read_csv('/content/drive/My Drive/Colab Notebooks/ML/Train_Data.csv')
# dataWithoutLabels = pd.read_csv(('/content/drive/My Drive/Colab Notebooks/ML/Test_Data.csv'))
# temp=pd.concat([dataset,dataWithoutLabels], ignore_index=True)
# print(dataset.tail)
# print(temp.columns.tolist())
#temp['Yearly Income in addition to Salary (e.g. Rental Income)']=pd.to_numeric(temp['Yearly Income in addition to Salary (e.g. Rental Income)'].map(lambda x: x.rstrip(' EUR')))
#print(temp['Yearly Income in addition to Salary (e.g. Rental Income)'].unique())

In [82]:
# col='Yearly Income in addition to Salary (e.g. Rental Income)'
# temp[col]=temp[col].map(lambda x: x.rstrip(' EUR'))
# print(temp[col].unique())
# a=temp[col].value_counts(dropna=False)
# print(a)
# X_train,X_test = temp.iloc[:1048574],temp.iloc[1048575:]
# print(X_train.head)
# rename_cols = {"Income in EUR":'Income'}
# train = train.rename(columns=rename_cols)
# for col in fill_col_dict.keys():
    # data[col] = data[col].fillna(fill_col_dict[col])
#X_train,X_test = data[features_col].iloc[:1048574],data[features_col].iloc[1048575:]

In [83]:

# to add noise on the data if dataframe has any NAN values
def add_noise(series, noise_level):
    return series * (1 + noise_level * np.random.randn(len(series)))


#Function to apply Target encoding on categorial data it takes train and test column along with target column to give logical mean value to strings.

def target_encode(trn_series, tst_series,target):
    min_samples_leaf=1
    smoothing=1
    noise_level=0
    temp = pd.concat([trn_series, target], axis=1)
    
    # Compute target mean 
    averages = temp.groupby(by=trn_series.name)[target.name].agg(["mean", "count"])
    
    # Compute smoothing
    smoothing = 1 / (1 + np.exp(-(averages["count"] - min_samples_leaf) / smoothing))
    
    # Apply average function to all target data
    prior = target.mean()
    
    # The bigger the count the less full_avg is taken into account
    averages[target.name] = prior * (1 - smoothing) + averages["mean"] * smoothing
    averages.drop(["mean", "count"], axis=1, inplace=True)
    
    # Apply averages to train and test series
    ft_trn_series = pd.merge(
        trn_series.to_frame(trn_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=trn_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    
    # pd.merge does not keep the index so restore it
    ft_trn_series.index = trn_series.index 
    ft_tst_series = pd.merge(
        tst_series.to_frame(tst_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=tst_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    
    # pd.merge does not keep the index so restore it
    ft_tst_series.index = tst_series.index
    
    return add_noise(ft_trn_series, noise_level), add_noise(ft_tst_series, noise_level)


In [84]:

def prepareData(dataset,dataWithoutLabels):
  #training data+test data
  
  dataset['Gender']=dataset['Gender'].replace(['unknown','0',np.nan],'UNKOWN')  
  dataset['Gender']=dataset['Gender'].replace(['f'],'female')  
  dataset['Housing Situation']=dataset['Housing Situation'].replace(['nA',0,'0'],'UNKOWN')
  dataset['Work Experience in Current Job [years]']=pd.to_numeric(dataset['Work Experience in Current Job [years]'].replace(['#NUM!'],0))
  dataset['Satisfation with employer']=dataset['Satisfation with employer'].replace([np.nan],'Somewhat Happy')
  dataset['Yearly Income in addition to Salary (e.g. Rental Income)']=pd.to_numeric(dataset['Yearly Income in addition to Salary (e.g. Rental Income)'].map(lambda x: x.rstrip(' EUR')))
  dataset['Hair Color']=dataset['Hair Color'].replace(['unknown','0',np.nan],'UNKOWN')
  #print('fillna mean')
  #dataset=dataset.fillna(dataset.mean())
  dataWithoutLabels['Gender']=dataWithoutLabels['Gender'].replace(['unknown','0',np.nan],'UNKOWN')  
  dataWithoutLabels['Gender']=dataWithoutLabels['Gender'].replace(['f'],'female')  
  dataWithoutLabels['Housing Situation']=dataWithoutLabels['Housing Situation'].replace(['nA',0,'0'],'UNKOWN')
  dataWithoutLabels['Work Experience in Current Job [years]']=pd.to_numeric(dataWithoutLabels['Work Experience in Current Job [years]'].replace(['#NUM!'],0))
  dataWithoutLabels['Satisfation with employer']=dataWithoutLabels['Satisfation with employer'].replace([np.nan],'Somewhat Happy')
  dataWithoutLabels['Yearly Income in addition to Salary (e.g. Rental Income)']=pd.to_numeric(dataWithoutLabels['Yearly Income in addition to Salary (e.g. Rental Income)'].map(lambda x: x.rstrip(' EUR')))
  dataWithoutLabels['Hair Color']=dataWithoutLabels['Hair Color'].replace(['unknown','0',np.nan],'UNKOWN')
  dataWithoutLabels=dataWithoutLabels.fillna(method='ffill')
  dataset=dataset.fillna(method='ffill')
    #encoding of categorical data
  dataset['Housing Situation'],dataWithoutLabels['Housing Situation']=target_encode(dataset['Housing Situation'], dataWithoutLabels['Housing Situation'],dataset['Total Yearly Income [EUR]'])
  dataset['Satisfation with employer'],dataWithoutLabels['Satisfation with employer']=target_encode(dataset['Satisfation with employer'], dataWithoutLabels['Satisfation with employer'],dataset['Total Yearly Income [EUR]'])
  dataset['Gender'],dataWithoutLabels['Gender']=target_encode(dataset['Gender'], dataWithoutLabels['Gender'],dataset['Total Yearly Income [EUR]'])
  dataset['Country'],dataWithoutLabels['Country']=target_encode(dataset['Country'], dataWithoutLabels['Country'],dataset['Total Yearly Income [EUR]'])
  dataset['Profession'],dataWithoutLabels['Profession']=target_encode(dataset['Profession'], dataWithoutLabels['Profession'],dataset['Total Yearly Income [EUR]'])
  dataset['University Degree'],dataWithoutLabels['University Degree']=target_encode(dataset['University Degree'], dataWithoutLabels['University Degree'],dataset['Total Yearly Income [EUR]'])
  dataset['Hair Color'],dataWithoutLabels['Hair Color']=target_encode(dataset['Hair Color'], dataWithoutLabels['Hair Color'],dataset['Total Yearly Income [EUR]'])
  return dataset,dataWithoutLabels


In [85]:

  # plot the Values on a graph
def plotActualVsPredictedIncome(y_test, y_pred):
    df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
    df1 = df.head(25)
    df1.plot(kind='bar',figsize=(10,8))
    plt.grid(which='major', linestyle='-', linewidth='0.5', color='green')
    plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
    plt.show()    

def trainModel(X_train, y_train, X_test, y_test):
   
    
    #Implementing linear regression (error rate 110000+)
    # trainingRegressor = LinearRegression()  
    # trainingRegressor.fit(X_train, y_train)
    # y_pred = trainingRegressor.predict(X_test)
    
    
    #Implementing RandomForest Regressor (error rate 80000+)
    #trainingRegressor = RandomForestRegressor(n_estimators=1000)
    #trainingRegressor.fit(X_train,y_train)
    #y_pred = trainingRegressor.predict(X_test)
    
    #Implementing lightgbm
    
    
    trainingRegressor = lgb.LGBMRegressor(num_leaves=300,
                                          learning_rate=0.05,
                                          n_estimators=2000)
    trainingRegressor.fit(X_train, y_train,
                          eval_set=[(X_test, y_test)],
                          eval_metric='l1',
                          early_stopping_rounds=5)

    
    print('Starting predicting...')
    # predict
    y_pred = trainingRegressor.predict(X_test, num_iteration=trainingRegressor.best_iteration_)
   
    #Implementing caatboost (error rate 60000+)
    #learning rate to get an optimal solution 
    
    # trainingRegressor = CatBoostRegressor(iterations=7000,learning_rate=0.02)
    # trainingRegressor.fit(X_train, y_train,use_best_model=True,verbose=True)
    # y_pred=trainingRegressor.predict(X_test)


    
    #evaluating the error
    df = pd.DataFrame({'Test': y_test, 'Prediction': y_pred})
    print("mean_absolute_error: %.2f"
      % mean_absolute_error(y_test, y_pred))
    
    #ploting the values on the graph
    #plotActualVsPredictedIncome(y_test, y_pred)

    return trainingRegressor
 

def runTrainedModelOnActualData(regressor, featuresToConsider,dataWithoutLabels):
    print(featuresToConsider)
    print('dataset',dataWithoutLabels[featuresToConsider].dtypes)
    #get the values for X_test    
    X = dataWithoutLabels[featuresToConsider].values
    print('head',X)
    #prediciting the values on the real test data
    predictedData = regressor.predict(X, num_iteration=regressor.best_iteration_)
    df = pd.DataFrame({'Predicted': predictedData})

    #write the data to csv
    submission_data = pd.read_csv('tcd-ml-1920-group-income-submission.csv')
    submission_data['Total Yearly Income [EUR]'] = predictedData
    submission_data.to_csv('tcd-ml-1920-group-income-submission.csv', encoding='utf-8', index = False)
    
    #fileName = 'submimssion.csv'
    #df.to_csv(fileName, sep=',',index=False)
    
    

    

#Reading data from CSV
dataset = pd.read_csv('tcd-ml-1920-group-income-train.csv')
dataWithoutLabels = pd.read_csv(('tcd-ml-1920-group-income-test.csv'))
#combined_data=pd.concat([dataset,dataWithoutLabels], ignore_index=True)

#preprocessing data on both the datasets



C:\Users\Spy\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\Spy\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [86]:

dataset,dataWithoutLabels = prepareData(dataset,dataWithoutLabels)


In [87]:
featuresToConsider = ['Year of Record', 'Housing Situation', 'Crime Level in the City of Employement', 'Work Experience in Current Job [years]', 'Satisfation with employer', 'Gender', 'Age', 'Country', 'Size of City', 'Profession', 'University Degree', 'Wears Glasses', 'Hair Color', 'Body Height [cm]', 'Yearly Income in addition to Salary (e.g. Rental Income)']

#getting thte target data
y = dataset['Total Yearly Income [EUR]'].values
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()

#Normalization
#dataset[featuresToConsider] = pd.DataFrame(scaler.fit_transform(dataset[featuresToConsider]), columns=featuresToConsider)
#dataWithoutLabels[featuresToConsider] = pd.DataFrame(scaler.fit_transform(dataWithoutLabels[featuresToConsider]), columns=featuresToConsider)
# print(dataset[featuresToConsider].dtypes)
# print(dataset[featuresToConsider].dtypes)
# print(dataset['Year of Record'].unique())

In [88]:

#Features Dataset
X = dataset[featuresToConsider].values

#divide the training data into test and train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

#get the trained model
regressor = trainModel(X_train, y_train, X_test, y_test)
# print(dataWithoutLabels.head)
# print(dataWithoutLabels.isnull().sum())
#run on the real test data using the. trained model
runTrainedModelOnActualData(regressor, featuresToConsider,dataWithoutLabels)

[1]	valid_0's l1: 69193.1	valid_0's l2: 1.14374e+10
Training until validation scores don't improve for 5 rounds
[2]	valid_0's l1: 65920.3	valid_0's l2: 1.04653e+10
[3]	valid_0's l1: 62823.7	valid_0's l2: 9.58619e+09
[4]	valid_0's l1: 59886.9	valid_0's l2: 8.78882e+09
[5]	valid_0's l1: 57112.7	valid_0's l2: 8.06547e+09
[6]	valid_0's l1: 54486.8	valid_0's l2: 7.41438e+09
[7]	valid_0's l1: 52002.5	valid_0's l2: 6.82261e+09
[8]	valid_0's l1: 49651.7	valid_0's l2: 6.28402e+09
[9]	valid_0's l1: 47429.4	valid_0's l2: 5.79565e+09
[10]	valid_0's l1: 45328.3	valid_0's l2: 5.3496e+09
[11]	valid_0's l1: 43343.4	valid_0's l2: 4.94521e+09
[12]	valid_0's l1: 41470.4	valid_0's l2: 4.57854e+09
[13]	valid_0's l1: 39706.2	valid_0's l2: 4.24865e+09
[14]	valid_0's l1: 38043.2	valid_0's l2: 3.94675e+09
[15]	valid_0's l1: 36469.2	valid_0's l2: 3.67194e+09
[16]	valid_0's l1: 34990	valid_0's l2: 3.42308e+09
[17]	valid_0's l1: 33590.7	valid_0's l2: 3.19486e+09
[18]	valid_0's l1: 32268	valid_0's l2: 2.9866e+09
[

[154]	valid_0's l1: 9778.95	valid_0's l2: 5.95991e+08
[155]	valid_0's l1: 9775.29	valid_0's l2: 5.95636e+08
[156]	valid_0's l1: 9768.93	valid_0's l2: 5.95112e+08
[157]	valid_0's l1: 9767.06	valid_0's l2: 5.94733e+08
[158]	valid_0's l1: 9763.37	valid_0's l2: 5.94405e+08
[159]	valid_0's l1: 9759.42	valid_0's l2: 5.9387e+08
[160]	valid_0's l1: 9753.83	valid_0's l2: 5.93143e+08
[161]	valid_0's l1: 9752.06	valid_0's l2: 5.93044e+08
[162]	valid_0's l1: 9743.11	valid_0's l2: 5.92386e+08
[163]	valid_0's l1: 9740.19	valid_0's l2: 5.92181e+08
[164]	valid_0's l1: 9732.92	valid_0's l2: 5.91683e+08
[165]	valid_0's l1: 9731.59	valid_0's l2: 5.9163e+08
[166]	valid_0's l1: 9727.51	valid_0's l2: 5.91312e+08
[167]	valid_0's l1: 9725.44	valid_0's l2: 5.91166e+08
[168]	valid_0's l1: 9721	valid_0's l2: 5.9066e+08
[169]	valid_0's l1: 9715.59	valid_0's l2: 5.90302e+08
[170]	valid_0's l1: 9711.04	valid_0's l2: 5.9007e+08
[171]	valid_0's l1: 9705.06	valid_0's l2: 5.89507e+08
[172]	valid_0's l1: 9702.33	valid_0

[314]	valid_0's l1: 9441.02	valid_0's l2: 5.67824e+08
[315]	valid_0's l1: 9440.37	valid_0's l2: 5.67821e+08
[316]	valid_0's l1: 9439.51	valid_0's l2: 5.67735e+08
[317]	valid_0's l1: 9438.13	valid_0's l2: 5.67681e+08
[318]	valid_0's l1: 9437.83	valid_0's l2: 5.67652e+08
[319]	valid_0's l1: 9436.15	valid_0's l2: 5.67583e+08
[320]	valid_0's l1: 9434.55	valid_0's l2: 5.67468e+08
[321]	valid_0's l1: 9433.28	valid_0's l2: 5.67389e+08
[322]	valid_0's l1: 9431.97	valid_0's l2: 5.67327e+08
[323]	valid_0's l1: 9431.32	valid_0's l2: 5.67304e+08
[324]	valid_0's l1: 9429.11	valid_0's l2: 5.67133e+08
[325]	valid_0's l1: 9428.05	valid_0's l2: 5.67038e+08
[326]	valid_0's l1: 9426.93	valid_0's l2: 5.66947e+08
[327]	valid_0's l1: 9426.76	valid_0's l2: 5.66921e+08
[328]	valid_0's l1: 9426.01	valid_0's l2: 5.66827e+08
[329]	valid_0's l1: 9424.68	valid_0's l2: 5.66775e+08
[330]	valid_0's l1: 9423.42	valid_0's l2: 5.66684e+08
[331]	valid_0's l1: 9421.61	valid_0's l2: 5.66597e+08
[332]	valid_0's l1: 9421.35	

[470]	valid_0's l1: 9316.67	valid_0's l2: 5.61601e+08
[471]	valid_0's l1: 9315.79	valid_0's l2: 5.61583e+08
[472]	valid_0's l1: 9315.21	valid_0's l2: 5.61583e+08
[473]	valid_0's l1: 9314.12	valid_0's l2: 5.61549e+08
[474]	valid_0's l1: 9313.5	valid_0's l2: 5.61513e+08
[475]	valid_0's l1: 9312.9	valid_0's l2: 5.61449e+08
[476]	valid_0's l1: 9312.72	valid_0's l2: 5.6145e+08
[477]	valid_0's l1: 9312.38	valid_0's l2: 5.61464e+08
[478]	valid_0's l1: 9311.95	valid_0's l2: 5.61464e+08
[479]	valid_0's l1: 9311.47	valid_0's l2: 5.6145e+08
[480]	valid_0's l1: 9311.21	valid_0's l2: 5.6141e+08
[481]	valid_0's l1: 9310.82	valid_0's l2: 5.61409e+08
[482]	valid_0's l1: 9310.16	valid_0's l2: 5.61353e+08
[483]	valid_0's l1: 9309.82	valid_0's l2: 5.61309e+08
[484]	valid_0's l1: 9309.15	valid_0's l2: 5.61291e+08
[485]	valid_0's l1: 9308.45	valid_0's l2: 5.61196e+08
[486]	valid_0's l1: 9308.15	valid_0's l2: 5.61208e+08
[487]	valid_0's l1: 9307.64	valid_0's l2: 5.61235e+08
[488]	valid_0's l1: 9307.58	valid